In [1]:
from cfgs.cfgs import *
from prompts.query_intent import *
from prompts.gen_response import *
from src.openai_call import *
from src.location import *
from src.real_estate import *
from src.query import *

import pandas as pd
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

legal_dong_pdf = pd.read_csv("/home/jovyan/project/data/data_go_kr/legal_dong_clean.csv")
legal_dong_pdf = legal_dong_pdf.fillna('')
legal_dong_pdf = legal_dong_pdf[legal_dong_pdf['dong_name'].str.endswith('동')]
legal_dong_pdf = legal_dong_pdf[legal_dong_pdf['sd_name'].isin(['서울특별시', '경기도'])]
legal_dong_pdf['bjd_code'] = legal_dong_pdf['bjd_code'].map(lambda x: int(str(x)[:5]))

In [3]:
query = "서광교파크스위첸 최근 2달 매매 정보 알려줘"

# 1-1) query -> structure with LLM
resp = gen_query_nlu_and_structure(client, query)
query_structure = extract_json_from_response(resp)

agent_resp = ''

if query_structure.get('intent') == '해당 없음':
    agent_resp = "죄송합니다, 해당 질문은 제가 답변하기 어려운 유형입니다. 부동산(아파트 혹은 오피스텔) 거래 정보에 대해 질문해주세요."
try:
    # 1-2) Fill bjd name
    ## 1-2-1) Show Candidate bjd name list
    sd_sgg_dong_lst = get_candidate_bjd_name_lst(query_structure, legal_dong_pdf) 

    if not sd_sgg_dong_lst:
        agent_resp = "죄송합니다, 정확한 지역(or건물)명을 입력해주세요. 현재 저희 서비스는 수도권(서울, 경기) 지역의 정보만 제공하고 있습니다."

    ###### (실제로는 여기서 사용자에게 후보를 보여주고 선택받는 로직 필요)
    bjd_name = sd_sgg_dong_lst[0] # 테스트를 위해 첫 번째 후보 자동 선택
    query_structure = set_bjd_name(query_structure, bjd_name)
    query_structure = get_bjd_code_v3(query_structure, legal_dong_pdf)

    # 데이터 호출 및 후처리
    total_real_estate_pdf = get_real_estate_pdf(query_structure)
    filtered_pdf = postprocess_real_estate(total_real_estate_pdf, query_structure)
    
    # 데이터가 없는 경우 처리
    if filtered_pdf.empty:
        agent_resp = "요청하신 조건에 맞는 거래 내역을 찾지 못했습니다."

    # 답변 생성
    res_data_json = prepare_data_for_llm_by_pyeong(query_structure['intent'], filtered_pdf)
    final_resp = gen_final_answer(client, query_structure, res_data_json)
    
    agent_resp, use_data, query_summary = final_resp, res_data_json, make_query_summary(query_structure), make_cache_key_name(query_structure)

except Exception as e:
    # 파이프라인 중간에 알 수 없는 오류 발생 시 처리
    print(f"오류 발생: {e}")
    agent_resp = "요청을 처리하는 중 오류가 발생했습니다. 잠시 후 다시 시도해주세요."

print(agent_resp)

서광교파크스위첸의 최근 2달 매매 정보는 다음과 같습니다:

### 16평형
- 거래금액: 55,900만 원, 전용면적: 52㎡, 층: 15
- 거래금액: 56,800만 원, 전용면적: 52㎡, 층: 19
- 거래금액: 55,500만 원, 전용면적: 52㎡, 층: 9

### 18평형
- 거래금액: 70,000만 원, 전용면적: 59㎡, 층: 21
- 거래금액: 69,200만 원, 전용면적: 59㎡, 층: 12
- 거래금액: 69,000만 원, 전용면적: 59㎡, 층: 9

### 25평형
- 거래금액: 88,000만 원, 전용면적: 84㎡, 층: 17
- 거래금액: 87,500만 원, 전용면적: 84㎡, 층: 19
- 거래금액: 86,500만 원, 전용면적: 84㎡, 층: 27

가장 최근 거래는 25평형으로, 88,000만 원에 17층에서 거래되었습니다.


In [5]:
query_structure

{'intent': '단일 매물 조회',
 'target': 'APT_TRADE',
 'deal_type': '매매',
 'sido': '경기',
 'sigu': '수원시장안구',
 'dong': '연무동',
 'housing_name': '서광교파크스위첸',
 'min_area': None,
 'max_area': None,
 'start_date': '2025-06-01',
 'end_date': '2025-07-29',
 'max_price': None,
 'min_price': None,
 'min_monthly_rent': None,
 'max_monthly_rent': None,
 'query_type': None,
 'query_text': '서광교파크스위첸 최근 2달 매매 정보 알려줘',
 'bjd_code_lst': [41111]}

In [14]:
filtered_pdf.to_csv(index=False)

'단지명,건축년도,계약일자,전용면적,층,법정동,거래금액,대표평형,평형대\n서광교파크스위첸,2023,2025-06-27,52,6,연무동,55000,16,16평형\n서광교파크스위첸,2023,2025-06-17,84,19,연무동,87500,25,25평형\n서광교파크스위첸,2023,2025-06-14,59,12,연무동,69200,18,18평형\n서광교파크스위첸,2023,2025-06-23,52,18,연무동,53000,16,16평형\n서광교파크스위첸,2023,2025-06-23,52,27,연무동,55500,16,16평형\n서광교파크스위첸,2023,2025-06-18,84,17,연무동,88000,25,25평형\n서광교파크스위첸,2023,2025-06-21,52,15,연무동,55900,16,16평형\n서광교파크스위첸,2023,2025-06-21,52,15,연무동,55900,16,16평형\n서광교파크스위첸,2023,2025-06-14,52,18,연무동,53000,16,16평형\n서광교파크스위첸,2023,2025-06-03,59,9,연무동,69000,18,18평형\n서광교파크스위첸,2023,2025-06-03,84,27,연무동,86500,25,25평형\n서광교파크스위첸,2023,2025-06-01,59,21,연무동,70000,18,18평형\n서광교파크스위첸,2023,2025-07-10,52,19,연무동,56800,16,16평형\n서광교파크스위첸,2023,2025-07-19,52,9,연무동,55500,16,16평형\n'